In [1]:
import findspark
findspark.init("/usr/hdp/2.6.5.0-292/spark2")
import pyspark
sc = pyspark.SparkContext()

In [2]:
import pandas as pd
import pyspark
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
from nltk.corpus import stopwords
import re as re

In [4]:
from pyspark.ml.feature import CountVectorizer , IDF

In [5]:
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel

In [6]:
sqlContext.read.csv('/user/maria_dev/data/newsarticles_article.csv', sep=',', escape='"', header=True, 
               inferSchema=True, multiLine=True).count()

465160

In [7]:
data = sqlContext.read.csv('/user/maria_dev/data/newsarticles_article.csv', sep=',', escape='"', header=False, 
               inferSchema=True, multiLine=True)
tag = sqlContext.read.csv('/user/maria_dev/data/tags.csv', sep=',',  header=True, inferSchema=True, multiLine=True).withColumnRenamed("article_id\r", "article_id")

In [8]:
#data.head()

In [9]:
# tag.columns  

In [10]:
#tag.head()

In [11]:
df = data.join(tag, data._c0 == tag.article_id )

In [12]:
# df.count()

In [13]:
# df =df.na.drop(subset=['category_id','_c4','_c5'])

In [14]:
#df.count()

In [15]:
df2 = df.sample(False, 0.2) 

In [16]:
dfsplit = df2.randomSplit([1.0, 2.0],24)

In [17]:
# dfsplit[0].count()

In [18]:
# dfsplit[1].count()

In [19]:
reviews_train = dfsplit[0].rdd.map(lambda x : str(x['_c4']) + str(x['_c5']))
reviews_test = dfsplit[1].rdd.map(lambda x : str(x['_c4']) + str(x['_c5']))

In [20]:
reviews_train.top(1)

['‘Who does that?’ victim asks of gunman who opened fire on park filled with kids - Chicago Sun-Times#  ‘Who does that?’ victim asks of gunman who opened fire on park filled with\nkids\n\nBY FRANK MAIN, TINA SFONDELES, FRAN SPIELMAN AND STEFANO ESPOSITO Staff\nReporters  September 20, 2013 8:23AM\n\n25184020\n\n13 INJURED IN SHOOTING\n\nPolice provided this information on the victims:\n\n◆ The 3-year-old boy was shot in the face, in critical condition at Mt. Sinai.\n\n◆ A 15-year-old boy was shot in the arm, in stable condition at Holy Cross.\n\n◆ A 17-year-old girl was shot in the foot, in stable condition at Holy Cross.\n\n◆ A 23-year-old woman was shot in the foot, in stable condition at St.\nAnthony.\n\n◆ A 31-year-old man was shot in the buttocks, in stable condition at\nNorthwestern.\n\n◆ A 28-year-old man was shot in the wrist, leg and hip, in unknown condition\nat Mt. Sinai.\n\n◆ A 41-year-old man was shot in the buttocks, in serious condition at Stroger.\n\n◆ A 26-year-old man

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
StopWords = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maria_dev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/maria_dev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
def tokenlize(doc):
    import nltk, re
    from nltk.corpus import stopwords
    from nltk.corpus import wordnet

    r = re.compile(r'[\w]+')
    my_stopwords = nltk.corpus.stopwords.words('english')
    porter = nltk.PorterStemmer()

    newdoc = []
    for word in nltk.regexp_tokenize(doc, r): 
        newWord = porter.stem(word.lower()) 
        if newWord in my_stopwords: 
            continue
        tokenSynsets = wordnet.synsets(newWord)
        newdoc.append(newWord if tokenSynsets == [] else tokenSynsets[0].lemma_names()[0])
    return newdoc

In [23]:
traintokens = reviews_train.map( lambda doc: tokenlize(doc))
testtokens = reviews_test.map( lambda doc: tokenlize(doc))

In [24]:
from pyspark.mllib.feature import HashingTF
hasingTF = HashingTF(2 ** 16)
trainTf = hasingTF.transform(traintokens)
trainTf.cache()
from pyspark.mllib.feature import IDF
idf = IDF().fit(trainTf)
trainTfidf = idf.transform(trainTf)
trainTfidf.cache()

MapPartitionsRDD[40] at mapPartitions at PythonMLLibAPI.scala:1339

In [25]:
from pyspark.mllib.regression import LabeledPoint
trainLabels = dfsplit[0].rdd.map(lambda x : x['category_id'])
train = trainLabels.zip(trainTfidf).map(lambda pair : LabeledPoint(pair[0], pair[1]))
train.cache()

PythonRDD[44] at RDD at PythonRDD.scala:48

In [26]:
from pyspark.mllib.feature import HashingTF
testTf = hasingTF.transform(testtokens )

In [27]:
testTfidf = idf.transform(testTf)

In [28]:
testLabels = dfsplit[1].rdd.map(lambda x : x['category_id'])
test = testLabels.zip(testTfidf).map(lambda pair : LabeledPoint(pair[0], pair[1]))
test.cache()

PythonRDD[52] at RDD at PythonRDD.scala:48

In [29]:
# testLabels.count()

In [30]:
# testTfidf.count()

In [31]:
# test.first()

In [32]:
testCount= test.count()

In [33]:
from pyspark.mllib.classification import NaiveBayes
model = NaiveBayes.train(train, 0.1)
print('model trained')

model trained


In [34]:
predictionAndLabel = test.map(lambda p: (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda x: x[0] == x[1]).count() / testCount
print(accuracy)

0.843298634582


In [35]:
from  pyspark.mllib.classification import LogisticRegressionWithLBFGS
lrModel = LogisticRegressionWithLBFGS.train(train, iterations=10, numClasses=20) 
print('model trained')

model trained


In [36]:
predictionAndLabel = test.map(lambda p: (lrModel.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda x: x[0] == x[1]).count() / testCount
print(accuracy )

0.862897120454
